## best score 0.87

In [13]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV

In [14]:
train = pd.read_csv('Data/selected_train.csv')
test = pd.read_csv('Data/selected_test.csv')
val = pd.read_csv('Data/selected_val.csv')

In [15]:
X_train = train.drop('NObeyesdad', axis=1)
y_train = train['NObeyesdad']
X_test = test.drop('NObeyesdad', axis=1)
y_test = test['NObeyesdad']
X_val = val.drop('NObeyesdad', axis=1)
y_val = val['NObeyesdad']

In [16]:
#some random parameters
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)
y_pred1 = rf.predict(X_val)
class_report1 = classification_report(y_val, y_pred1)
print("Classification Report:\n", class_report1)
conf_matrix1 = confusion_matrix(y_val, y_pred1)
print("\nConfusion Matrix:")
print(conf_matrix1)

Classification Report:
               precision    recall  f1-score   support

           0       0.93      0.88      0.90       525
           1       0.80      0.86      0.83       624
           2       0.83      0.81      0.82       604
           3       0.94      0.95      0.95       714
           4       1.00      1.00      1.00       837
           5       0.71      0.69      0.70       532
           6       0.71      0.72      0.72       523

    accuracy                           0.86      4359
   macro avg       0.85      0.84      0.84      4359
weighted avg       0.86      0.86      0.86      4359


Confusion Matrix:
[[461  58   0   0   0   4   2]
 [ 31 536   3   0   0  46   8]
 [  2   1 489  31   4  25  52]
 [  0   0  31 678   0   0   5]
 [  2   0   0   1 834   0   0]
 [  0  65  15   0   0 368  84]
 [  0  12  51   9   0  73 378]]


In [17]:
clf = RandomForestClassifier(random_state=42)

RandomForestClassifier( )

param_grid = {
    'n_estimators':  [ 300,400,450, 500],
    'criterion' : ['gini', 'entropy'],
    'max_depth': [20, 25, 30],
    'n_jobs':[-1], 
    'min_samples_split':[ 2, 3,4, 5, 6,7,],
    'max_features': ['auto', 'sqrt', 'log2']

}

grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)


best_params = grid_search.best_params_
best_score = grid_search.best_score_
cv_results = grid_search.cv_results_

print("Best Parameters:", best_params)
print("Best Score:", best_score)
print("Best model CV Results:", cv_results["mean_test_score"][grid_search.best_index_])

Fitting 3 folds for each of 432 candidates, totalling 1296 fits
[CV] END criterion=gini, max_depth=20, max_features=auto, min_samples_split=2, n_estimators=400, n_jobs=-1; total time=   0.0s
[CV] END criterion=gini, max_depth=20, max_features=auto, min_samples_split=2, n_estimators=450, n_jobs=-1; total time=   0.0s
[CV] END criterion=gini, max_depth=20, max_features=auto, min_samples_split=2, n_estimators=300, n_jobs=-1; total time=   0.0s
[CV] END criterion=gini, max_depth=20, max_features=auto, min_samples_split=2, n_estimators=400, n_jobs=-1; total time=   0.0s
[CV] END criterion=gini, max_depth=20, max_features=auto, min_samples_split=2, n_estimators=400, n_jobs=-1; total time=   0.0s
[CV] END criterion=gini, max_depth=20, max_features=auto, min_samples_split=2, n_estimators=450, n_jobs=-1; total time=   0.0s
[CV] END criterion=gini, max_depth=20, max_features=auto, min_samples_split=2, n_estimators=300, n_jobs=-1; total time=   0.0s
[CV] END criterion=gini, max_depth=20, max_feat

/opt/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:547: FitFailedWarning: 
432 fits failed out of a total of 1296.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
174 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/anaconda3/lib/python3.11/site-packages/sklearn/base.py", line 1467, in wrapper
    estimator._validate_params()
  File "/opt/anaconda3/lib/python3.11/site-packages/sklearn/base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "/opt/anaconda3/lib/python3.11/site-

Best Parameters: {'criterion': 'entropy', 'max_depth': 20, 'max_features': 'sqrt', 'min_samples_split': 6, 'n_estimators': 300, 'n_jobs': -1}
Best Score: 0.8795599666011523
Best model CV Results: 0.8795599666011523


BEST:  
Best Parameters: {'criterion': 'entropy', 'max_depth': 20, 'max_features': 'sqrt', 'min_samples_split': 6, 'n_estimators': 300, 'n_jobs': -1}  
Best Score: 0.8795599666011523  
Best model CV Results: 0.8795599666011523

In [19]:
#best
best = RandomForestClassifier(criterion='entropy', max_depth=20,min_samples_split=6,n_estimators=300,n_jobs=-1 , random_state=42, max_features='sqrt')
best.fit(X_train, y_train)
y_pred1 = best.predict(X_val)
class_report1 = classification_report(y_val, y_pred1)
print("Classification Report:\n", class_report1)
conf_matrix1 = confusion_matrix(y_val, y_pred1)
print("\nConfusion Matrix:")
print(conf_matrix1)

Classification Report:
               precision    recall  f1-score   support

           0       0.93      0.90      0.91       525
           1       0.81      0.88      0.84       624
           2       0.84      0.81      0.83       604
           3       0.94      0.95      0.95       714
           4       1.00      1.00      1.00       837
           5       0.74      0.70      0.72       532
           6       0.73      0.74      0.73       523

    accuracy                           0.87      4359
   macro avg       0.86      0.85      0.85      4359
weighted avg       0.87      0.87      0.87      4359


Confusion Matrix:
[[471  49   0   0   0   3   2]
 [ 31 548   2   0   0  36   7]
 [  2   1 491  32   4  25  49]
 [  0   0  27 681   0   0   6]
 [  1   0   1   1 834   0   0]
 [  0  64  14   0   0 374  80]
 [  0  13  47   8   0  70 385]]
